In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

In [16]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [17]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [18]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/Users/pirmatov/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [20]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [21]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [26]:
%%time

factors = [64, 128]
regularization = [0.01, 0.05, 0.08]
iteration = [15, 30, 60]
grid_result = {}
for f in factors:
    for r in regularization:
        for it in iteration:
            print(f"factors: {f}, reg: {r}, iterations: {it}")
            model = AlternatingLeastSquares(factors=64, 
                                            regularization=0.05,
                                            iterations=15, 
                                            calculate_training_loss=True, 
                                            num_threads=10)

            model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=True)


            result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

            res = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
            print(res)
            print()
            print()
            grid_result[f"factors: {f}, reg: {r}, iterations: {it}"] = res
            
            

factors: 64, reg: 0.01, iterations: 15


  0%|          | 0/15 [00:00<?, ?it/s]

0.17649422400803352


factors: 64, reg: 0.01, iterations: 30


  0%|          | 0/15 [00:00<?, ?it/s]

0.17338021094926948


factors: 64, reg: 0.01, iterations: 60


...

0.17408337518834519


factors: 128, reg: 0.08, iterations: 30


  0%|          | 0/15 [00:00<?, ?it/s]

0.16875941737819955


factors: 128, reg: 0.08, iterations: 60


  0%|          | 0/15 [00:00<?, ?it/s]

0.17046710195881243


CPU times: user 22min 54s, sys: 3min 30s, total: 26min 25s
Wall time: 5min 59s


In [27]:
grid_result

{'factors: 64, reg: 0.01, iterations: 15': 0.17649422400803352,
 'factors: 64, reg: 0.01, iterations: 30': 0.17338021094926948,
 'factors: 64, reg: 0.01, iterations: 60': 0.1755901557006506,
 'factors: 64, reg: 0.05, iterations: 15': 0.17749874434957072,
 'factors: 64, reg: 0.05, iterations: 30': 0.1733802109492695,
 'factors: 64, reg: 0.05, iterations: 60': 0.17227523857357868,
 'factors: 64, reg: 0.08, iterations: 15': 0.17569060773480416,
 'factors: 64, reg: 0.08, iterations: 30': 0.17438473129080598,
 'factors: 64, reg: 0.08, iterations: 60': 0.17538925163234295,
 'factors: 128, reg: 0.01, iterations: 15': 0.1756906077348041,
 'factors: 128, reg: 0.01, iterations: 30': 0.171471622300349,
 'factors: 128, reg: 0.01, iterations: 60': 0.17398292315419164,
 'factors: 128, reg: 0.05, iterations: 15': 0.1736815670517305,
 'factors: 128, reg: 0.05, iterations: 30': 0.17106981416373446,
 'factors: 128, reg: 0.05, iterations: 60': 0.1771973882471093,
 'factors: 128, reg: 0.08, iterations: 15

In [31]:
import operator
max(grid_result.items(), key=operator.itemgetter(1))

('factors: 64, reg: 0.05, iterations: 15', 0.17749874434957072)